# Synthetic neural benchmarking

- this notebook will demo how to use CEBRA, piVAE, tSNE and UMAP on synthetic datasets.

- COLAB, Mac pre-M1, *Windows, and Ubuntu users can skip to cell 2.
- *This has not been tested on Windows

In [ ]:
#Attention: M1 chip users ONLY:
# TF is a dependency of piVAE, which is used in this demo.
# It requires a few extra steps that are addressed in this cell; in short, please run this once.
# See also: https://developer.apple.com/metal/tensorflow-plugin/ if any issues.

!pip uninstall -y tensorflow-deps tensorflow-macos tensorflow-metal keras

!conda install pytorch torchvision torchaudio -c pytorch
!pip install --upgrade --force-reinstall scikit-learn

!conda install -c apple -y tensorflow-deps==2.5.0 --force-reinstall
!python -m pip install tensorflow-macos
!python -m pip install tensorflow-metal

- Install note: be sure you have demo dependencies installed to use this notebook:

In [ ]:
!pip install --pre 'cebra[datasets,demos]'

In [ ]:
#import CEBRA:
import cebra.datasets
from cebra import CEBRA
from cebra.datasets import get_datapath

In [ ]:
# ONLY for using for t-sne, umap, and piVAE:
!pip install openTSNE
!pip install umap-learn
!git clone --depth 1 --filter=blob:none --sparse https://github.com/AdaptiveMotorControlLab/CEBRA.git && cd CEBRA && git sparse-checkout set third_party
!pip install tensorflow keras

In [ ]:
#import other packages:
import matplotlib.pyplot as plt
import numpy as np
import torch
import joblib as jl
import sklearn.linear_model
import openTSNE
import umap
import keras

In [ ]:
#import piVAE:
import os
import sys
sys.path.insert(0, '/content/CEBRA/third_party')


###### Update the imports in the script // can be dropped if https://github.com/AdaptiveMotorControlLab/CEBRA/pull/164 merged!
import re
file_path = '/content/CEBRA/third_party/pivae/pivae_code/pi_vae.py'
with open(file_path, 'r') as file:
    filedata = file.read()

filedata = re.sub(r'from keras', r'from tensorflow.keras', filedata)

with open(file_path, 'w') as file:
    file.write(filedata)
###########

# import
import pivae.pivae_code.pi_vae as pivae
from tensorflow.keras.callbacks import ModelCheckpoint

### Let's load the data:

- The data will be automatically downloaded into a `/data` folder.

In [ ]:
dataset = cebra.datasets.init("continuous-label-poisson")

In [ ]:
data = dataset.data
plt.scatter(data['z'][:, 0], data['z'][:, 1], c=data['u'], s=1, cmap='cool')
plt.axis('off')

### Define the reconstruction score we use for all methods:

In [ ]:
def reconstruction_score(x, y):

    def _linear_fitting(x, y):
        lin_model = sklearn.linear_model.LinearRegression()
        lin_model.fit(x, y)
        return lin_model.score(x, y), lin_model.predict(x)

    return _linear_fitting(x, y)

# CEBRA

- Define & fit a 🦓 <font color='blue'>C</font><font color='MediumSlateBlue'>E</font><font color='BlueViolet'>B</font><font color='MediumVioletRed'>R</font><font color='red'>A</font>  model:
- For a quick CPU run-time demo, you can drop `max_iterations` to 500; otherwise set to 10,000.

In [ ]:
max_iterations = 500

In [ ]:
cebra_model = CEBRA(
    model_architecture="offset1-model-mse",
    batch_size=512,
    learning_rate=1e-4,
    max_iterations=max_iterations,
    delta=0.1,
    conditional='delta',
    output_dimension=2,
    distance='euclidean',
    device="cuda_if_available",
    verbose=True,
)

In [ ]:
cebra_model.fit(data['x'][:12000], data['u'][:12000])

In [ ]:
cebra_output = cebra_model.transform(data['x'])
cebra_score, transformed_cebra_z = reconstruction_score(cebra_output,
                                                        data['z'][:, :2])
print(f"linear reconstruction score: {cebra_score}")
plt.scatter(transformed_cebra_z[:, 0],
            transformed_cebra_z[:, 1],
            c=data['u'],
            s=1,
            cmap='cool')
plt.axis('off')

# t-SNE

In [ ]:
tsne_model = openTSNE.TSNE(perplexity=84,
                           n_components=2,
                           initialization='pca',
                           random_state=None,
                           metric='euclidean')

In [ ]:
tsne_model.fit(data['x'])
tsne_output = tsne_model.fit(data['x'])

In [ ]:
tsne_score, transformed_tsne_z = reconstruction_score(tsne_output,
                                                      data['z'][:, :2])
print(f"linear reconstruction score: {tsne_score}")
plt.scatter(transformed_tsne_z[:, 0],
            transformed_tsne_z[:, 1],
            c=data['u'],
            s=1,
            cmap='cool')
plt.axis('off')

# UMAP

In [ ]:
umap_model = umap.UMAP(n_neighbors=68,
                       min_dist=0.2475,
                       n_components=2,
                       random_state=None,
                       metric='euclidean')
umap_output = umap_model.fit_transform(data['x'])

In [ ]:
umap_score, transformed_umap_z = reconstruction_score(umap_output,
                                                      data['z'][:, :2])
print(f"linear reconstruction score: {umap_score}")
plt.scatter(transformed_umap_z[:, 0],
            transformed_umap_z[:, 1],
            c=data['u'],
            s=1,
            cmap='cool')
plt.axis('off')

# piVAE

The dataset parsing, model configuration and training are all adapted from https://github.com/zhd96/pi-vae/blob/main/examples/pi-vae_simulated_data_continuous_label.ipynb
- piVAE has a long run time. We store checkpoints.

In [ ]:
u_true = data['u']
z_true = data['z']
x_true = data['x']
x_all = x_true.reshape(50, 300, -1)
u_all = u_true.reshape(50, 300, -1)

x_train = x_all[:40]
u_train = u_all[:40]
x_valid = x_all[40:]
u_valid = u_all[40:]
print(f'Train set has {len(x_train)} samples')

In [ ]:
def custom_data_generator(x_train, u_train):
    while True:
        for i in range(len(x_train)):
            yield [x_train[i], u_train[i]], x_train[i]  # or appropriate target


In [ ]:
vae = pivae.vae_mdl(dim_x=x_all[0].shape[-1],
                    dim_z=2,
                    dim_u=u_all[0].shape[-1],
                    gen_nodes=60,
                    n_blk=2,
                    mdl='poisson',
                    disc=False,
                    learning_rate=5e-4)

In [ ]:
model_chk_path = 'synthetic_pivae.h5'

mcp = ModelCheckpoint(model_chk_path,
                      monitor="val_loss",
                      save_best_only=True,
                      save_weights_only=True)

s_n = vae.fit(pivae.custom_data_generator(x_train, u_train),
                        steps_per_epoch=len(x_train),
                        epochs=100,
                        verbose=0,
                        validation_data=pivae.custom_data_generator(
                            x_valid, u_valid),
                        validation_steps=len(x_valid),
                        callbacks=[mcp])

In [ ]:
vae.load_weights(model_chk_path)
outputs = vae.predict_generator(pivae.custom_data_generator(x_all, u_all),
                                steps=len(x_all))
#post_mean, post_log_var, z_sample,fire_rate, lam_mean, lam_log_var, z_mean, z_log_var
z_post = outputs[0]

In [ ]:
pivae_score, transformed_pivae_z = reconstruction_score(z_post,
                                                        data['z'][:, :2])
print(f"linear reconstruction score: {pivae_score}")
plt.scatter(transformed_pivae_z[:, 0],
            transformed_pivae_z[:, 1],
            c=data['u'],
            s=1,
            cmap='cool')
plt.axis('off')